In [2]:
import pandas as pd
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, BatchNormalization,Input
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import np_utils
import keras.backend as Kr
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight
import numpy as np
from numpy import exp
# Library for Gaussian process
# import GPy
##Library for visualization
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib;matplotlib.rcParams['figure.figsize'] = (10,7)
import pylab 
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import GPy

Using TensorFlow backend.


In [3]:
df_loc = pd.read_csv("2D_biv_matern_8100_projection.csv", sep = ",")

In [4]:
df_loc.head()

,x,y,var1,var2,proj_var1,proj_var2,threshold
0,0.000000,0.0,-3.561195,-3.635512,-3.501161,-3.703366,4
1,0.011236,0.0,-3.664613,-3.773622,-3.627711,-3.815330,4
2,0.022472,0.0,-3.541629,-3.837566,-3.590461,-3.782373,4
3,0.033708,0.0,-3.881213,-4.192305,-3.956989,-4.106658,4
4,0.044944,0.0,-4.151834,-4.490577,-4.101776,-4.549881,3


In [5]:
df_train,df_test = train_test_split(df_loc, test_size = 0.1)

In [6]:
model_list05 = list()
model_list25 = list()
model_list50 = list()
model_list75 = list()
model_list95 = list()

In [7]:
# def model_function(q):
#     def tilted_loss(y,f):

#         e1 = (y[:,0]-f[:,0])
#         e2 = (y[:,1]-f[:,1])
#         return Kr.mean(Kr.maximum(q*e1, (q-1)*e1), axis=-1) + Kr.mean(Kr.maximum(q*e2, (q-1)*e2), axis=-1)

#     # DeepKriging model for continuous data
#     model = Sequential()
#     # model.add(Dense(100, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
#     model.add(Dense(50, input_dim = phi.shape[1],  
#                     kernel_initializer='he_uniform', activation='relu'))
#     # model.add(Dropout(rate=0.5))
#     # model.add(BatchNormalization())
#     model.add(Dense(50, activation='relu'))
#     model.add(Dense(30, activation='relu'))
#     #     model.add(Dense(100, activation='relu'))
#     # model.add(Dense(100, activation='relu'))
#     model.add(Dense(10, activation='relu'))
#     model.add(Dense(10, activation='relu'))
#     #     model.add(Dense(50, activation='relu'))
#     #     model.add(Dense(50, activation='relu'))
#     #     model.add(Dense(50, activation='relu'))
#     #     model.add(Dense(10, activation='relu'))
#     #model.add(Dense(50, activation='relu'))
#     #model.add(Dropout(rate=0.5))
#     #     model.add(Dense(10, activation='relu'))
#     #model.add(BatchNormalization())
#     # model.add(Dense(10, activation='relu'))
#     model.add(Dense(2, activation='linear'))
#     #     quantile = 0.5
#     optimizer = keras.optimizers.Adam(lr=0.001)
#     model.compile(loss=tilted_loss, optimizer=optimizer)
#     #     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

#     callbacks = [EarlyStopping(monitor='val_loss', patience=200),
#                  ModelCheckpoint(filepath='projection_model.h5', monitor='val_loss', save_best_only=True)]

#     result = model.fit(phi, y, callbacks=callbacks,
#                        validation_split = 0.1, epochs = 500, batch_size = 64, verbose = 2)

#     model = keras.models.load_model('projection_model.h5', custom_objects={'tilted_loss':
#                                                                            tilted_loss})
#     return model

In [8]:
for h in range(7):
    df_train1 = df_train.loc[df_train['threshold'] == h+1]
    print(df_train1.head(1))
    y = np.array(df_train1[["proj_var1","proj_var2"]])
    N = len(df_train1)
    s = np.vstack((df_train1["x"],df_train1["y"])).T

    num_basis = [2**2,2**2,4**2]
    knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
    ##Wendland kernel
    K = 0
    phi = np.zeros((N, sum(num_basis)))

    for res in range(len(num_basis)):
        theta = 1/np.sqrt(num_basis[res])*2.5
        knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
        knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
        for i in range(num_basis[res]):
            d = np.linalg.norm(s-knots[i,:],axis=1)/theta
            for j in range(len(d)):
                if d[j] >= 0 and d[j] <= 1:
                    phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
                else:
                    phi[j,i + K] = 0
        K = K + num_basis[res]
    def model_function(q):
        def tilted_loss(y,f):

            e1 = (y[:,0]-f[:,0])
            e2 = (y[:,1]-f[:,1])
            return Kr.mean(Kr.maximum(q*e1, (q-1)*e1), axis=-1) + Kr.mean(Kr.maximum(q*e2, (q-1)*e2), axis=-1)

        # DeepKriging model for continuous data
        model = Sequential()
        # model.add(Dense(100, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
        model.add(Dense(50, input_dim = phi.shape[1],  
                        kernel_initializer='he_uniform', activation='relu'))
        # model.add(Dropout(rate=0.5))
        # model.add(BatchNormalization())
        model.add(Dense(50, activation='relu'))
        model.add(Dense(30, activation='relu'))
        #     model.add(Dense(100, activation='relu'))
        # model.add(Dense(100, activation='relu'))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(10, activation='relu'))
        #     model.add(Dense(50, activation='relu'))
        #     model.add(Dense(50, activation='relu'))
        #     model.add(Dense(50, activation='relu'))
        #     model.add(Dense(10, activation='relu'))
        #model.add(Dense(50, activation='relu'))
        #model.add(Dropout(rate=0.5))
        #     model.add(Dense(10, activation='relu'))
        #model.add(BatchNormalization())
        # model.add(Dense(10, activation='relu'))
        model.add(Dense(2, activation='linear'))
        #     quantile = 0.5
        optimizer = keras.optimizers.Adam(lr=0.001)
        model.compile(loss=tilted_loss, optimizer=optimizer)
        #     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        callbacks = [EarlyStopping(monitor='val_loss', patience=200),
                     ModelCheckpoint(filepath='projection_model.h5', monitor='val_loss', save_best_only=True)]
        print("running model for threshold "+str(h+1)+ " at quantile level "+str(q)+".")
        result = model.fit(phi, y, callbacks=callbacks,
                           validation_split = 0.1, epochs = 500, batch_size = 64, verbose = 0)

        model = keras.models.load_model('projection_model.h5', custom_objects={'tilted_loss':
                                                                               tilted_loss})
        return model
    model_list05.append(model_function(0.05))
    model_list25.append(model_function(0.25))
    model_list50.append(model_function(0.50))
    model_list75.append(model_function(0.75))
    model_list95.append(model_function(0.95))
    

             x         y      var1     var2  proj_var1  proj_var2  threshold
7775  0.393258  0.966292 -7.559454 -7.58313  -7.575446  -7.559317          1
running model for threshold 1 at quantile level 0.05.
running model for threshold 1 at quantile level 0.25.
running model for threshold 1 at quantile level 0.5.
running model for threshold 1 at quantile level 0.75.
running model for threshold 1 at quantile level 0.95.
             x         y      var1      var2  proj_var1  proj_var2  threshold
1613  0.932584  0.191011 -3.581529 -4.407575  -3.545473  -4.456143          2
running model for threshold 2 at quantile level 0.05.
running model for threshold 2 at quantile level 0.25.
running model for threshold 2 at quantile level 0.5.
running model for threshold 2 at quantile level 0.75.
running model for threshold 2 at quantile level 0.95.
             x         y      var1      var2  proj_var1  proj_var2  threshold
5293  0.820225  0.651685 -4.573466 -4.871032   -4.53551  -4.915997        

In [9]:
N = len(df_test)
s = np.vstack((df_test["x"],df_test["y"])).T

num_basis = [2**2,5**2,9**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi_test = np.zeros((N, sum(num_basis)))

for res in range(len(num_basis)):
    
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi_test[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi_test[j,i + K] = 0
    K = K + num_basis[res]
y_test = np.array(df_test[["var1","var2"]])




print(df_train.head(1))
y = np.array(df_train[["var1","var2"]])
N = len(df_train)
s = np.vstack((df_train["x"],df_train["y"])).T

num_basis = [2**2,5**2,9**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))

for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]

# DeepKriging model for continuous data
model = Sequential()
# model.add(Dense(100, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(100, input_dim = phi.shape[1],  
                kernel_initializer='he_uniform', activation='relu'))
# model.add(Dropout(rate=0.5))
# model.add(BatchNormalization())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
#     model.add(Dense(100, activation='relu'))
# model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
#     model.add(Dense(50, activation='relu'))
#     model.add(Dense(50, activation='relu'))
#     model.add(Dense(10, activation='relu'))
#model.add(Dense(50, activation='relu'))
#model.add(Dropout(rate=0.5))
#     model.add(Dense(10, activation='relu'))
#model.add(BatchNormalization())
model.add(Dense(10, activation='relu'))
model.add(Dense(2, activation='linear'))
#     quantile = 0.5
optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)
#     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='val_loss', patience=200),
             ModelCheckpoint(filepath='mean_model.h5', monitor='val_loss', save_best_only=True)]
print("running model for mean. ")
result = model.fit(phi, y, callbacks=callbacks,
                   validation_data = (phi_test,y_test), epochs = 500, batch_size = 128, verbose = 2)

model_mean = model #keras.models.load_model('mean_model.h5')



             x         y      var1      var2  proj_var1  proj_var2  threshold
5293  0.820225  0.651685 -4.573466 -4.871032   -4.53551  -4.915997          3
running model for mean. 
Train on 7290 samples, validate on 810 samples
Epoch 1/500
 - 1s - loss: 9.5357 - val_loss: 2.2138
Epoch 2/500
 - 0s - loss: 0.8857 - val_loss: 0.3792
Epoch 3/500
 - 0s - loss: 0.3609 - val_loss: 0.3214
Epoch 4/500
 - 0s - loss: 0.3271 - val_loss: 0.2993
Epoch 5/500
 - 0s - loss: 0.3030 - val_loss: 0.2758
Epoch 6/500
 - 0s - loss: 0.2790 - val_loss: 0.2531
Epoch 7/500
 - 0s - loss: 0.2435 - val_loss: 0.2072
Epoch 8/500
 - 0s - loss: 0.1761 - val_loss: 0.1386
Epoch 9/500
 - 0s - loss: 0.1301 - val_loss: 0.1173
Epoch 10/500
 - 0s - loss: 0.1147 - val_loss: 0.1095
Epoch 11/500
 - 0s - loss: 0.1026 - val_loss: 0.0952
Epoch 12/500
 - 0s - loss: 0.0962 - val_loss: 0.0951
Epoch 13/500
 - 0s - loss: 0.0904 - val_loss: 0.0842
Epoch 14/500
 - 0s - loss: 0.0832 - val_loss: 0.0778
Epoch 15/500
 - 0s - loss: 0.0779 - val

Epoch 151/500
 - 0s - loss: 0.0276 - val_loss: 0.0366
Epoch 152/500
 - 0s - loss: 0.0271 - val_loss: 0.0357
Epoch 153/500
 - 0s - loss: 0.0280 - val_loss: 0.0352
Epoch 154/500
 - 0s - loss: 0.0269 - val_loss: 0.0369
Epoch 155/500
 - 0s - loss: 0.0281 - val_loss: 0.0347
Epoch 156/500
 - 0s - loss: 0.0270 - val_loss: 0.0382
Epoch 157/500
 - 0s - loss: 0.0270 - val_loss: 0.0366
Epoch 158/500
 - 0s - loss: 0.0282 - val_loss: 0.0330
Epoch 159/500
 - 0s - loss: 0.0274 - val_loss: 0.0351
Epoch 160/500
 - 0s - loss: 0.0279 - val_loss: 0.0434
Epoch 161/500
 - 0s - loss: 0.0291 - val_loss: 0.0372
Epoch 162/500
 - 0s - loss: 0.0275 - val_loss: 0.0352
Epoch 163/500
 - 0s - loss: 0.0285 - val_loss: 0.0369
Epoch 164/500
 - 0s - loss: 0.0265 - val_loss: 0.0357
Epoch 165/500
 - 0s - loss: 0.0273 - val_loss: 0.0362
Epoch 166/500
 - 0s - loss: 0.0277 - val_loss: 0.0410
Epoch 167/500
 - 0s - loss: 0.0264 - val_loss: 0.0355
Epoch 168/500
 - 0s - loss: 0.0266 - val_loss: 0.0360
Epoch 169/500
 - 1s - loss: 

Epoch 303/500
 - 0s - loss: 0.0235 - val_loss: 0.0335
Epoch 304/500
 - 0s - loss: 0.0214 - val_loss: 0.0327
Epoch 305/500
 - 0s - loss: 0.0213 - val_loss: 0.0322
Epoch 306/500
 - 0s - loss: 0.0217 - val_loss: 0.0388
Epoch 307/500
 - 0s - loss: 0.0222 - val_loss: 0.0329
Epoch 308/500
 - 0s - loss: 0.0229 - val_loss: 0.0339
Epoch 309/500
 - 0s - loss: 0.0224 - val_loss: 0.0340
Epoch 310/500
 - 0s - loss: 0.0211 - val_loss: 0.0322
Epoch 311/500
 - 0s - loss: 0.0213 - val_loss: 0.0322
Epoch 312/500
 - 0s - loss: 0.0218 - val_loss: 0.0334
Epoch 313/500
 - 0s - loss: 0.0216 - val_loss: 0.0364
Epoch 314/500
 - 0s - loss: 0.0226 - val_loss: 0.0385
Epoch 315/500
 - 0s - loss: 0.0228 - val_loss: 0.0324
Epoch 316/500
 - 0s - loss: 0.0207 - val_loss: 0.0339
Epoch 317/500
 - 0s - loss: 0.0211 - val_loss: 0.0315
Epoch 318/500
 - 0s - loss: 0.0228 - val_loss: 0.0315
Epoch 319/500
 - 0s - loss: 0.0215 - val_loss: 0.0315
Epoch 320/500
 - 0s - loss: 0.0214 - val_loss: 0.0325
Epoch 321/500
 - 0s - loss: 

Epoch 455/500
 - 0s - loss: 0.0190 - val_loss: 0.0306
Epoch 456/500
 - 0s - loss: 0.0185 - val_loss: 0.0303
Epoch 457/500
 - 0s - loss: 0.0192 - val_loss: 0.0303
Epoch 458/500
 - 0s - loss: 0.0190 - val_loss: 0.0332
Epoch 459/500
 - 0s - loss: 0.0186 - val_loss: 0.0302
Epoch 460/500
 - 0s - loss: 0.0190 - val_loss: 0.0309
Epoch 461/500
 - 0s - loss: 0.0194 - val_loss: 0.0316
Epoch 462/500
 - 0s - loss: 0.0199 - val_loss: 0.0360
Epoch 463/500
 - 0s - loss: 0.0201 - val_loss: 0.0309
Epoch 464/500
 - 0s - loss: 0.0189 - val_loss: 0.0316
Epoch 465/500
 - 0s - loss: 0.0187 - val_loss: 0.0298
Epoch 466/500
 - 0s - loss: 0.0190 - val_loss: 0.0283
Epoch 467/500
 - 0s - loss: 0.0188 - val_loss: 0.0317
Epoch 468/500
 - 0s - loss: 0.0183 - val_loss: 0.0308
Epoch 469/500
 - 0s - loss: 0.0182 - val_loss: 0.0309
Epoch 470/500
 - 0s - loss: 0.0217 - val_loss: 0.0329
Epoch 471/500
 - 0s - loss: 0.0194 - val_loss: 0.0324
Epoch 472/500
 - 0s - loss: 0.0190 - val_loss: 0.0312
Epoch 473/500
 - 0s - loss: 

In [10]:
pred = model_mean.predict(phi_test)
df_test["var1_mean"] = pred[:,0]
df_test["var2_mean"] = pred[:,1]

 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
np.mean((df_test["var2"]-df_test["var2_mean"])**2)

0.01812480712699123

In [12]:
df_test.head(1
            )

,x,y,var1,var2,proj_var1,proj_var2,threshold,var1_mean,var2_mean
6163,0.483146,0.764045,-6.200776,-4.984735,-6.357624,-4.786358,7,-6.380405,-4.999733


In [13]:
a = np.array([-2.47176554,-1.82406163,-1.08755835,-0.60572381,0.03170641,0.26759939,0.24032180])
b = np.array([0.6715845,0.7423779,0.8441032,0.8847470,0.9492417,0.9276952,0.7906538])

df_test.reset_index()
for i in range(len(df_test)):
    vec_distance = abs(df_test.iloc[i]["var2_mean"] - b*df_test.iloc[i]["var1_mean"] - a)/np.sqrt(1+b**2)
    index = np.where(vec_distance == min(vec_distance))
    
#     print(index[0])
#     print(df_test.iloc[i]['threshold'])
    df_test.iloc[i]['threshold'] = index[0] + 1
#     print(df_test.iloc[i]['threshold'])
    

 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
for h in range(3):
    df_test1 = df_test.loc[df_test['threshold'] == h+1]
#     print(df_test1.head())
    N = len(df_test1)
    s = np.vstack((df_test1["x"],df_test1["y"])).T

    num_basis = [2**2,2**2,4**2]
    knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
    ##Wendland kernel
    K = 0
    phi = np.zeros((N, sum(num_basis)))

    for res in range(len(num_basis)):
        theta = 1/np.sqrt(num_basis[res])*2.5
        knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
        knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
        for i in range(num_basis[res]):
            d = np.linalg.norm(s-knots[i,:],axis=1)/theta
            for j in range(len(d)):
                if d[j] >= 0 and d[j] <= 1:
                    phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
                else:
                    phi[j,i + K] = 0
        K = K + num_basis[res]
    print(h)
    model = model_list05[h]
    pred = model.predict(phi)
    df_test1["proj_var1_pred05"] = pred[:,0]
    df_test1["proj_var2_pred05"] = pred[:,1]
    model = model_list25[h]
    pred = model.predict(phi)
    df_test1["proj_var1_pred25"] = pred[:,0]
    df_test1["proj_var2_pred25"] = pred[:,1]
    model = model_list50[h]
    pred = model.predict(phi)
    df_test1["proj_var1_pred50"] = pred[:,0]
    df_test1["proj_var2_pred50"] = pred[:,1]
    model = model_list75[h]
    pred = model.predict(phi)
    df_test1["proj_var1_pred75"] = pred[:,0]
    df_test1["proj_var2_pred75"] = pred[:,1]
    model = model_list95[h]
    pred = model.predict(phi)
    df_test1["proj_var1_pred95"] = pred[:,0]
    df_test1["proj_var2_pred95"] = pred[:,1]
    if(h == 0):
        new_df_test = df_test1 
    else:
        print(h)
        new_df_test = pd.concat([new_df_test,df_test1], axis = 0)

# N = len(df_test)
# s = np.vstack((df_test["x"],df_test["y"])).T

# num_basis = [2**2,5**2,9**2]
# knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
# ##Wendland kernel
# K = 0
# phi = np.zeros((N, sum(num_basis)))

# for res in range(len(num_basis)):
    
#     theta = 1/np.sqrt(num_basis[res])*2.5
#     knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
#     knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
#     for i in range(num_basis[res]):
#         d = np.linalg.norm(s-knots[i,:],axis=1)/theta
#         for j in range(len(d)):
#             if d[j] >= 0 and d[j] <= 1:
#                 phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
#             else:
#                 phi[j,i + K] = 0
#     K = K + num_basis[res]

# pred = model_mean.predict(phi_test)
# new_df_test["var1_mean"] = pred[:,0]
# new_df_test["var2_mean"] = pred[:,1]
    

0


 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

1


 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

1
2
2


 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [1]:
new_df_test.head()


NameError: name 'new_df_test' is not defined

In [16]:
new_df_test.to_csv("median_results.csv")

In [17]:
df_train.to_csv("train_data.csv")